## **Setup colab**

In [ ]:
# #Download data from google drive
!git clone https://github.com/schonkopf/soundscape_IR.git #copy package HarryLab built from github
def gdrive_load_all_files(folder_id,file_type='.csv'): #only download csv file
  from soundscape_IR.soundscape_viewer import gdrive_handle #using function from harryLab package
  Gdrive=gdrive_handle(folder_id=folder_id)
  Gdrive.list_query(file_extension=file_type)
  for n in range(len(Gdrive.file_list)):
    Gdrive.file_list[n].GetContentFile(Gdrive.file_list[n]['title'])

# #upload data to gdrive
from google.colab import drive
drive.mount('/content/gdrive/')
# #左側> file> gdrive> 找存檔位置> copy path> 要存檔的時候把path 附上去


Cloning into 'soundscape_IR'...
remote: Enumerating objects: 2443, done.
remote: Counting objects: 100% (543/543), done.
remote: Compressing objects: 100% (257/257), done.
remote: Total 2443 (delta 345), reused 474 (delta 286), pack-reused 1900 (from 1)
Receiving objects: 100% (2443/2443), 74.02 MiB | 13.03 MiB/s, done.
Resolving deltas: 100% (1605/1605), done.
Mounted at /content/gdrive/


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Data- only Digestion genes

## **Data orgization**

In [ ]:
# Load data
#gdrive_load_all_files(folder_id='1EWajc53WcY126yKmEMhMarXIswrrKpTe',file_type='.csv')
data=pd.read_csv('1_without3d_onlyDigesition.csv', header=0)
# data['Label'] = [data['Treatment'][i]+'_'+str(data['Age'][i])+'d'+'_'+str(data['Number'][i]) for i in range(len(data['Treatment']))]
data.set_index('Sample', inplace=True)
data.index.name = None

data.head(5)

,Age,Treatment,Replicate,trypsin-2,aminopeptidase N,carboxypeptidase B,"β-1,3-glucanase",α-amylase 1,"endo-β-1,4-glucanase D-like",pancreatic triacylglycerol lipase,gastric triacylglycerol lipase,phospholipase B1
2LF_1,2,500,1,0.000001,0.0043,0.000320,0.000060,0.000113,0.000111,0.000021,0.000729,0.000972
2LF_2,2,500,2,0.000004,0.0030,0.000061,0.000049,0.000084,0.000564,0.000014,0.000590,0.000958
2LF_3,2,500,3,0.000043,0.0040,0.000290,0.000014,0.000099,0.000729,0.000013,0.000441,0.000498
2LF_4,2,500,4,0.000018,0.0038,0.000180,0.000051,0.000089,0.000568,0.000017,0.000600,0.000700
4LF_1,4,500,1,0.011000,0.0120,0.012000,0.000477,0.000505,0.000069,0.000570,0.001413,0.002326


In [ ]:
# Min-Max rescale
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
data_nor=preprocessing.normalize(data.iloc[:,3:],norm='max',axis=0) #Max: Max-min rescale in range 0-1, axis=1=rescale by raw, axis=0=rescale by column
"""
data.iloc[:,3:] = 只拿第四行之後的ｄａｔａ分析
"""

df_data_nor=pd.DataFrame(data_nor)

#
Sample=data.index
Feature=data.iloc[:,3:].columns.values
df_data_nor.index=Sample
df_data_nor.columns=Feature

##放入Series
df_data_nor.insert(0, 'Num', data['Replicate'].values)
df_data_nor.insert(0, 'Age', data['Age'].values)
df_data_nor.insert(0, 'Treatment', data['Treatment'].values)
# df_data_nor.to_csv('/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/0_df_data_nor.csv')

df_data_nor.head(3)

,Treatment,Age,Num,trypsin-2,aminopeptidase N,carboxypeptidase B,"β-1,3-glucanase",α-amylase 1,"endo-β-1,4-glucanase D-like",pancreatic triacylglycerol lipase,gastric triacylglycerol lipase,phospholipase B1
2LF_1,500,2,1,0.000018,0.047778,0.006531,0.034639,0.113813,0.096888,0.004745,0.221031,0.191071
2LF_2,500,2,2,0.000048,0.033333,0.001245,0.028458,0.084350,0.493098,0.003129,0.178849,0.188455
2LF_3,500,2,3,0.000544,0.044444,0.005918,0.008249,0.099419,0.637151,0.002787,0.133642,0.097855


In [ ]:
##定義treatments
Treatment=np.array(data.iloc[:,1],dtype=str)
Age=np.array(data.iloc[:,0],dtype=str)
Num=np.array(data.iloc[:,2],dtype=str)
Label=np.array(data['Treatment'][i]+'_'+str(data['Age'][i])+'d'+'_'+str(data['Replicate'][i]) for i in range(len(data['Treatment'])))

## **PCA: data_nor**


In [ ]:
## PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=9)
pca_result = pca.fit_transform(df_data_nor.iloc[:,3:])
pca_explained_var=pca.explained_variance_ratio_


## Get observation score
df_PCA= pd.DataFrame(data = pca_result
             , columns = ["PC 1 ("+str(np.round(pca_explained_var[0]*100,2))+"%)","PC 2 ("+str(np.round(pca_explained_var[1]*100,2))+"%)",
                     "PC 3 ("+str(np.round(pca_explained_var[2]*100,2))+"%)","PC 4 ("+str(np.round(pca_explained_var[3]*100,2))+"%)",
                     "PC 5 ("+str(np.round(pca_explained_var[4]*100,2))+"%)","PC 6 ("+str(np.round(pca_explained_var[5]*100,2))+"%)"
                     "PC 7 ("+str(np.round(pca_explained_var[6]*100,2))+"%)",
                     "PC 8 ("+str(np.round(pca_explained_var[7]*100,2))+"%)",
                     "PC 9 ("+str(np.round(pca_explained_var[8]*100,2))+"%)"])
#
df_PCA['Treatment']=Treatment
df_PCA['Age']=Age
df_PCA['Num']=Num


## Get feature loadings
# FeatureLoading = pca.components_.T
FeatureLoading = pca.components_.T * np.sqrt(pca.explained_variance_)
df_FeatureLoading = pd.DataFrame(FeatureLoading, columns=['PC1', 'PC2','PC3', 'PC4','PC5','PC6','PC7', 'PC8','PC9'], index=df_data_nor.iloc[:,3:].columns)

## save file
# df_PCA.to_csv('/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/2_PCA_data_nor_ComponentScore_240904.csv')
# df_FeatureLoading.to_csv('/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/2.PCA_data_nor_FeatureLoading_240904.csv')


ValueError: Shape of passed values is (64, 8), indices imply (64, 7)

In [ ]:
df_PCA.head(3)
## "PC" values= 每一個應變數在principal component的貢獻度（the scores of each observation）

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

PC1_score = df_PCA.loc[:,["PC 1 ("+str(np.round(pca_explained_var[0]*100,2))+"%)",'Treatment','Age','Num']]
PC1_score['AgeNum'] = PC1_score['Age'].astype(str)+'d_'+PC1_score['Num'].astype(str)
PC1_score=PC1_score.loc[:,["PC 1 ("+str(np.round(pca_explained_var[0]*100,2))+"%)",'Treatment','Age','AgeNum']].pivot(index='Treatment', columns='AgeNum')["PC 1 ("+str(np.round(pca_explained_var[0]*100,2))+"%)"]
fig = px.imshow(PC1_score, aspect="auto", color_continuous_scale='viridis')
fig.update_layout(width=1000, height=250)

fig.write_html("/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/2.PCAscore_nor_Heatmap_PC1_240904.html")


fig.show()

In [ ]:
PC2_score = df_PCA.loc[:,["PC 2 ("+str(np.round(pca_explained_var[1]*100,2))+"%)",'Treatment','Age','Num']]
PC2_score['AgeNum'] = PC2_score['Age'].astype(str)+'d_'+PC2_score['Num'].astype(str)
PC2_score=PC2_score.loc[:,["PC 2 ("+str(np.round(pca_explained_var[1]*100,2))+"%)",'Treatment','Age','AgeNum']].pivot(index='Treatment', columns='AgeNum')["PC 2 ("+str(np.round(pca_explained_var[1]*100,2))+"%)"]
fig = px.imshow(PC2_score, aspect="auto", color_continuous_scale='viridis')
fig.update_layout(width=1200, height=250)

fig.write_html("/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/2.PCAscore_nor_Heatmap_PC2_240904.html")


fig.show()

In [ ]:
PC3_score = df_PCA.loc[:,["PC 3 ("+str(np.round(pca_explained_var[2]*100,2))+"%)",'Treatment','Age','Num']]
PC3_score['AgeNum'] = PC3_score['Age'].astype(str)+'d_'+PC3_score['Num'].astype(str)
PC3_score=PC3_score.loc[:,["PC 3 ("+str(np.round(pca_explained_var[2]*100,2))+"%)",'Treatment','Age','AgeNum']].pivot(index='Treatment', columns='AgeNum')["PC 3 ("+str(np.round(pca_explained_var[2]*100,2))+"%)"]
fig = px.imshow(PC3_score, aspect="auto", color_continuous_scale='viridis')
fig.update_layout(width=1000, height=250)

fig.write_html("/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/2.PCAscore_nor_Heatmap_PC3_240904.html")


fig.show()

In [ ]:
df_FeatureLoading.head(3)
##"PC" values= 每一個自變數對principal component造成的影響（the loadings (or weights) of each original variable

,PC1,PC2,PC3,PC4,PC5
trypsin-2,-0.157841,-0.023462,-0.064779,0.064652,0.091629
aminopeptidase N,-0.059032,0.106707,0.067921,-0.016545,0.024413
carboxypeptidase B,-0.144091,0.093108,-0.017324,0.066086,-0.079431


In [ ]:
fig = px.imshow(df_FeatureLoading.loc[:,['PC1','PC2','PC3']], aspect="auto", y=df_FeatureLoading.index, color_continuous_scale='viridis')
fig.update_layout(width=400, height=400)

fig.write_html("/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/2.FeatureLoading_nor_Heatmap_240904.html")

fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Define custom color map for Age groups
color_map = {'2': 'gold', '4': 'orange', '5': 'lightcoral', '6': 'tomato', '7': 'lightskyblue', '8': 'blue', '9': 'gray', '10': 'black'}

color_map1 = {'2': 'DarkSlateGrey', '4': 'royalblue', '5': 'skyblue', '6': 'yellowgreen', '7': 'turquoise', '8': 'orange', '9': 'gold', '10': 'yellow'}

color_map2 = {'2': 'Darkred', '4': 'red', '5': 'salmon', '6': 'orange', '7': 'gold', '8': 'yellowgreen', '9': 'lightseagreen', '10': 'cyan'}

color_map3 = {'2': 'black', '4': 'blue', '5': 'darkviolet', '6': 'fuchsia', '7': 'crimson', '8': 'coral', '9': 'orange', '10': 'gold'}

color_map4 = {'2': 'DarkSlateGrey', '4': 'blue', '5': 'deepskyblue', '6': 'paleturquoise', '7': 'lightcoral', '8': 'orange', '9': 'gold', '10': 'yellow'}

color_map5 = {'2': 'DarkSlateGrey', '4': 'blue', '5': 'deepskyblue', '6': 'paleturquoise', '7': 'crimson', '8': 'lightpink', '9': 'orange', '10': 'gold'}

color_map6 = {'2': 'black', '4': 'blue', '5': 'lightskyblue', '6': 'aqua', '7': 'green', '8': 'limegreen', '9': 'yellowgreen', '10': 'gold'}

color_map7 = {'2': 'black', '4': 'Darkred', '5': 'red', '6': 'peru', '7': 'lightcoral', '8': 'orange', '9': 'gold', '10': 'yellow'}


# Create scatter plot for PCA scores
fig = px.scatter(df_PCA, x="PC 1 ("+str(np.round(pca_explained_var[0]*100,2))+"%)",
                 y="PC 2 ("+str(np.round(pca_explained_var[1]*100,2))+"%)",
                 color="Age", color_discrete_map=color_map5, symbol= "Treatment", symbol_sequence= ['circle-open','asterisk-open'], hover_data=['Treatment', 'Age', 'Num' ])




# Add feature loading vectors as arrows
# for i, feature in enumerate(df_data_nor.columns[3:]):
#   fig.add_trace(go.Scatter(x=[0, FeatureLoading[i, 0]], y=[0, FeatureLoading[i, 1]],
#                     mode='lines', name=feature,
#                     marker=dict(size=10, color='black'), line=dict(width=2)))
  # Add text annotation for feature name
  # fig.add_annotation(x=FeatureLoading[i, 0], y=FeatureLoading[i, 1],
  #                    ax=0, ay=0, showarrow=True,xanchor="center", yanchor="bottom",text=feature,
  #                    yshift=5,)
  # Add arrorhead QQQQQQ 畫不出箭頭
  # fig.add_annotation(x=FeatureLoading[i, 0], y=FeatureLoading[i, 1],
                     #  ax=0, ay=0, showarrow=True, arrowsize=1.5, arrowwidth=1, arrowhead=1)



fig.update_traces(textposition='top center')
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.update_layout(legend_orientation="v")
fig.update_layout(height=700,width=800)
#fig.update_xaxes(rangeselector_font_size=20,tickfont_size=20)
#fig.update_yaxes(tickfont_size=20)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.update_xaxes(showline=True, linewidth=3, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=3, linecolor='black', mirror=True)
fig.update_traces(marker=dict(size=10, line=dict(width=2, color='black')))
fig.update_traces(opacity=1)


## save file
fig.write_html("/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/3_PCA_nor_PC1+2_240904_color_map5.html")


fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Define custom color map for Age groups
color_map = {'2': 'gold', '4': 'orange', '5': 'lightcoral', '6': 'tomato', '7': 'lightskyblue', '8': 'blue', '9': 'gray', '10': 'black'}
color_map5 = {'2': 'DarkSlateGrey', '4': 'blue', '5': 'deepskyblue', '6': 'paleturquoise', '7': 'crimson', '8': 'lightpink', '9': 'orange', '10': 'gold'}


# Create scatter plot for PCA scores
fig = px.scatter(df_PCA, x="PC 1 ("+str(np.round(pca_explained_var[0]*100,2))+"%)",
                 y="PC 3 ("+str(np.round(pca_explained_var[2]*100,2))+"%)",
                 color="Age", color_discrete_map=color_map5, symbol= "Treatment", symbol_sequence= ['circle-open','asterisk-open'], hover_data=['Treatment', 'Age', 'Num' ])




# Add feature loading vectors as arrows
# for i, feature in enumerate(df_data_nor.columns[3:]):
#   fig.add_trace(go.Scatter(x=[0, FeatureLoading[i, 0]], y=[0, FeatureLoading[i, 1]],
#                     mode='lines', name=feature,
#                     marker=dict(size=10, color='black'), line=dict(width=2)))
  # Add text annotation for feature name
  # fig.add_annotation(x=FeatureLoading[i, 0], y=FeatureLoading[i, 1],
  #                    ax=0, ay=0, showarrow=True,xanchor="center", yanchor="bottom",text=feature,
  #                    yshift=5,)
  # Add arrorhead QQQQQQ 畫不出箭頭
  # fig.add_annotation(x=FeatureLoading[i, 0], y=FeatureLoading[i, 1],
                     #  ax=0, ay=0, showarrow=True, arrowsize=1.5, arrowwidth=1, arrowhead=1)



fig.update_traces(textposition='top center')
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.update_layout(legend_orientation="v")
fig.update_layout(height=700,width=800)
#fig.update_xaxes(rangeselector_font_size=20,tickfont_size=20)
#fig.update_yaxes(tickfont_size=20)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.update_xaxes(showline=True, linewidth=3, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=3, linecolor='black', mirror=True)
fig.update_traces(marker=dict(size=10, line=dict(width=2, color='black')))
fig.update_traces(opacity=1)


## save file
fig.write_html("/content/gdrive/MyDrive/Colab Notebooks/2024_Jasper_PCA/2_PCAResult_OnlyDigestion/3_PCA_nor_PC1+3_240904_color_map5.html")


fig.show()